In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'..')


from food.paths import *
from food.search import *
from mytools.tools import *
import pandas as pd
from pathlib import Path
import numpy as np
from IPython.display import clear_output

photos_path = test_path/'test_photos'

dev


/home/dima/new/mytools/mytools/visual.py:26: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  series2tensor = lambda series:torch.tensor([np.array(c) for c in series.values])


In [2]:
from segmentor.segment import get_segment_model

In [3]:
domain

'https://dima.skynet.center'

In [4]:
df = pd.read_csv(test_path/'test_set.csv')
df.columns = ['name','fat','protein','carb','energy','','','','','','composition','url']
df = df[[c for c in df.columns if c!='']]

df['url'] = df['url'].apply(lambda n:f'{domain}/test_photos/{n}.jpeg')

for c in ['fat','protein','carb','energy']:
    df[c] = pd.to_numeric(df[c].str.replace(',','.'))

In [5]:
model_path = checkpoints_path.ls()[0]
segment_model = get_segment_model(model_path)

/home/dima/new/segmentator/mmseg/models/builder.py:59: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


load pre-trained weight from imagenet21k


In [6]:
preds = []

In [7]:
for i in df.index[:50]:
    clear_output()
    print(i)
    
    # try:
    s = df.loc[i]
    img,clip_df,masks,urls,stats = search(s['url'],segment_model=segment_model,prompt_factor=0.1,exand_times =0,stego = False)
    pred = stats.T[['energy','protein','carb','fat']]
    pred.index = [i]
    preds.append(pred)
    # except:
    #     pass

preds = pd.concat(preds)

preds.columns = [c+'_pred' for c in preds.columns]

preds = df.join(preds).dropna()

(preds['energy']/preds['energy_pred']-1).abs().mean()

preds['error'] = (preds['energy']/preds['energy_pred']-1)

22


RuntimeError: CUDA out of memory. Tried to allocate 10.88 GiB (GPU 0; 23.70 GiB total capacity; 1.00 GiB already allocated; 6.94 GiB free; 1.23 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [20]:
url

NameError: name 'url' is not defined